https://stepik.org/lesson/1576195/step/8

In [ ]:
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim

class FuncModel(nn.Module):
    def __init__(self):
        super().__init__()
        # модель однослойной полносвязной нейронной сети:
        # 1-й слой: число входов 5 (x, x^2, x^3, x^4, x^5), число нейронов 1
        self.layer = nn.Linear(5, 1)

    def forward(self, x):
        x.unsqueeze_(-1)
        xx = torch.cat([x, x ** 2, x ** 3, x ** 4, x ** 5], dim=1)
        y = self.layer(xx)
        return y


torch.manual_seed(1)

model = FuncModel()

epochs = 20 # число эпох обучения
batch_size = 16 # размер батча

# данные обучающей выборки (значения функции)
data_x = torch.arange(-5, 5, 0.05) #тензоры data_x, data_y не менять
data_y = torch.sin(2 * data_x) - 0.3 * torch.cos(8 * data_x) + 0.1 * data_x ** 2

ds = data.TensorDataset(data_x, data_y) # создание dataset
d_train, d_val = data.random_split(ds, [0.7, 0.3])
train_data = data.DataLoader(d_train, batch_size=batch_size, shuffle=True)
train_data_val = data.DataLoader(d_val, batch_size=batch_size, shuffle=False)

optimizer = optim.RMSprop(params=model.parameters(), lr=0.01)
loss_func = nn.MSELoss()

loss_lst_val = []   # список значений потерь при валидации
loss_lst = []       # список значений потерь при обучении

for _e in range(epochs):
    model.train()
    loss_mean = 0 # вспомогательные переменные для вычисления среднего значения потерь при обучении
    lm_count = 0

    for x_train, y_train in train_data:
        predict = model(x_train).squeeze()
        loss = loss_func(predict, y_train)

        optimizer.zero_grad()       # очистка градиента
        loss.backward()             # обратное распространение ошибки
        optimizer.step()            # обновление параметров сети
        # loss_mean += loss.item()    # накопление потерь по батчам
        # lm_count += 1

        # вычисление среднего значения функции потерь по всей выборке
        lm_count += 1
        loss_mean = 1 / lm_count * loss.item() + (1 - 1 / lm_count) * loss_mean

    # валидация модели
    model.eval()
    Q_val = 0
    count_val = 0

    for x_val, y_val in train_data_val:
        with torch.no_grad():
            predict = model(x_val).squeeze()
            loss = loss_func(predict, y_val)
            Q_val += loss.item()
            count_val += 1

    # сохранить средние потери, вычисленные по выборке валидации, в переменной Q_val
    Q_val /= count_val

    loss_lst.append(loss_mean)
    loss_lst_val.append(Q_val)

model.eval()
# выполнить прогноз модели по всем данным выборки (ds.data)
preds = model(data_x)
# вычислить потери с помощью loss_func по всем данным выборки ds; значение Q сохранить в виде вещественного числа
Q = loss_func(preds.flatten(), data_y).item()

# print(Q)
# print(loss_lst)

[58000.0390625, 3705.9929131401914, 430.9513530731201, 85.20848337809245, 34.44961049821642, 23.521960682339138, 19.50560885005528, 18.379872534010147, 18.04986233181424, 17.378559960259334, 17.50553406609429, 17.526691648695202, 17.30922285715739, 17.085827933417427, 16.803658379448787, 17.058402379353844, 16.40661409166124, 15.860542880164251, 16.22151358922323, 15.998286777072483]
